In [1]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

llm = ChatOllama(model="llama3.2:1b")

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
import faiss

class DataIngestion:
    def __init__(self, file_path):
        self.file_path = file_path
        self.vstore = None
    async def vectorDB(self):
        """
        Creates a vector store that can store document embeddings a svectors
        """
        loader = CSVLoader(file_path=self.file_path)
        docs = await loader.alazy_load()

        #splitter
        text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunck_overlap=200)
        doc_splits = text_spliter.split_documents(docs)

        #Embeddings
        ollama_embedding = OllamaEmbeddings(model="llama3.2:1b")

        #Index creation
        vector_indexes = faiss.IndexFlatL2(ollama_embedding.embed_query(" "))

        #Create FAISS vectorstore
        self.vstore = FAISS(
            embedding_function= ollama_embedding,
            index= vector_indexes,
            docstore= InMemoryDocstore(),
            index_to_docstore_id= {},
        )
        #Add documnts to vector store
        _ = await self.vstore.aadd_documents(doc_splits)

        return self.vstore

In [ ]:
async def retriever(file_path: str):
    """
    Creates a retriever using Data ingestion class and vectorDB method
    """
    ingestion_obj = DataIngestion(file_path= file_path)
    vdb = ingestion_obj.vectorDB()
    retriever = vdb.as_retriever()
    return retriever

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

class RAGAgent:
    def __init__(self, llm, query):
        self.retriever = retriever()
        self.llm = llm
        self.query = query

    async def rag_chain_response(self):
        """
        creates an a flow to retrieve similar documents for search query and pass it to llam as context for better response
        """
        system_prompt = (
            "You are an assistant for question-answering tasks. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know. Use three sentences maximum and keep the "
            "answer concise."
            "\n\n"
            "{context}"
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("user", "{self.query}")
            ]
        )

        qa_chain = create_stuff_documents_chain(self.llm, prompt)
        rag_chain = create_retrieval_chain(self.retriever, qa_chain)
        response =await  rag_chain.ainvoke(self.query)
        return response



In [ ]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    query = input("Enter yor query: ")
    rag_obj = RAGAgent(llm, query)
    response = rag_obj.rag_chain_response()
    print(response)